In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(OPENAI_API_KEY[:2])
print(UPSTAGE_API_KEY[30:])
print(TAVILY_API_KEY[:2])

sk
YN
tv


In [9]:
import os
import json
from pprint import pprint
from typing import List, Dict, Any

# LangChain 및 관련 라이브러리 임포트
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.schema import Document
from langchain_community.vectorstores import FAISS # 'langchain_community'로 임포트 변경
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import chain
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

# Upstage 임베딩 임포트
from langchain_upstage import UpstageEmbeddings 

# Wikipedia 임포트
from wikipedia import summary as wiki_summary_func
from wikipedia import set_lang as wiki_set_lang

# ----------------------------------------------------------------------
# 0. 환경 변수 및 초기 설정
# ----------------------------------------------------------------------

# ⚠️ 실행 전 다음 환경 변수를 설정해야 합니다.
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY" 
# os.environ["UPSTAGE_API_KEY"] = "YOUR_UPSTAGE_API_KEY" 

# API 키 확인
openai_key = os.getenv("OPENAI_API_KEY")
upstage_key = os.getenv("UPSTAGE_API_KEY")

if not openai_key or not upstage_key:
    raise ValueError("OPENAI_API_KEY 또는 UPSTAGE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 위키피디아 언어 설정 (한국어)
wiki_set_lang("ko")



In [10]:

# ----------------------------------------------------------------------
# 1. 카페 메뉴 데이터 및 벡터 DB 구축 함수
# ----------------------------------------------------------------------

def build_cafe_vector_db():
    """카페 메뉴 텍스트를 로드하고 벡터 DB(FAISS)를 구축합니다."""
    print(">>> 1단계: 벡터 DB 구축 시작...")
    
    # 1-1. 메뉴 데이터 (Document 객체로 변환)
    menu_data_str = """
    메뉴: 아메리카노, 가격: ₩4,500, 재료: 에스프레소, 뜨거운 물, 설명: 원두 본연의 깔끔하고 시원한 맛. 산미와 바디감이 적절함.
    메뉴: 카페 라떼, 가격: ₩5,000, 재료: 에스프레소, 우유, 설명: 우유의 고소함과 에스프레소의 조화가 부드러움.
    메뉴: 바닐라 라떼, 가격: ₩5,500, 재료: 에스프레소, 우유, 바닐라 시럽, 설명: 달콤한 바닐라 향이 추가된 부드러운 라떼.
    메뉴: 초코 프라페, 가격: ₩6,500, 재료: 초콜릿 소스, 우유, 얼음, 휘핑크림, 설명: 진한 초코맛을 시원하게 즐기는 음료.
    메뉴: 딸기 스무디, 가격: ₩6,000, 재료: 냉동 딸기, 요거트, 우유, 설명: 신선한 딸기의 상큼함이 가득한 스무디.
    메뉴: 레몬 에이드, 가격: ₩5,500, 재료: 레몬즙, 탄산수, 설탕, 설명: 상큼하고 청량감 있는 수제 에이드.
    메뉴: 얼그레이, 가격: ₩4,000, 재료: 얼그레이 잎, 뜨거운 물, 설명: 베르가못 향이 은은한 클래식 홍차.
    메뉴: 허니 브레드, 가격: ₩7,000, 재료: 식빵, 버터, 꿀, 생크림, 설명: 달콤하고 부드러운 디저트, 커피와 완벽한 조화.
    메뉴: 치즈 케이크, 가격: ₩6,000, 재료: 크림치즈, 비스킷, 설명: 꾸덕하고 진한 뉴욕 스타일 치즈 케이크.
    메뉴: 마카롱 세트, 가격: ₩12,000, 재료: 아몬드 가루, 설탕, 필링(랜덤), 설명: 다양한 맛의 꼬끄와 필링이 어우러진 선물 세트.
    """
    
    docs = [Document(page_content=line.strip(), metadata={"source": "cafe_menu"}) 
            for line in menu_data_str.strip().split('\n') if line.strip()]
    
    # 텍스트 분할 (청크 사이즈는 작게 설정)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    split_docs = text_splitter.split_documents(docs)

    # 1-2. 임베딩 모델 설정 (Upstage 모델 사용)
    embeddings = UpstageEmbeddings(
        model="solar-embedding-1-large",
        api_key=upstage_key
    )

    # 1-3. FAISS를 사용한 벡터 인덱스 생성
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    
    print(">>> 벡터 DB 구축 완료.")
    return vectorstore

# 벡터 DB 구축 및 검색기 생성
cafe_vectorstore = build_cafe_vector_db()
cafe_retriever = cafe_vectorstore.as_retriever(search_kwargs={"k": 3})



>>> 1단계: 벡터 DB 구축 시작...
>>> 벡터 DB 구축 완료.


In [11]:

# ----------------------------------------------------------------------
# 2. 2개의 도구를 정의하고 LLM에 바인딩
# ----------------------------------------------------------------------

# a) wiki_summary (위키피디아 요약 도구)
@tool
def wiki_summary(topic: str) -> str:
    """위키피디아에서 일반 지식을 검색하고 요약합니다. 예를 들어 '커피의 역사'나 '음료 제조 방법'에 사용합니다."""
    print(f"\n[Tool Call] 위키피디아 검색 실행: {topic}")
    try:
        return wiki_summary_func(topic, sentences=3)
    except Exception:
        return f"위키피디아에서 '{topic}'에 대한 정보를 찾지 못했습니다."


# b) db_search_cafe_func (로컬 DB 검색 도구)
@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 가격, 재료, 설명 등 메뉴 관련 정보를 검색합니다. '아메리카노 가격'과 같은 질문에 사용합니다."""
    print(f"\n[Tool Call] 로컬 메뉴 DB 검색 실행: {query}")
    docs = cafe_retriever.get_relevant_documents(query)
    
    if docs:
        # Document 객체를 문자열로 변환하여 반환
        result_texts = [f"메뉴 정보: {doc.page_content}" for doc in docs]
        return "\n".join(result_texts)
    else:
        return "요청하신 메뉴 정보를 DB에서 찾을 수 없습니다."

# LLM 설정 및 도구 바인딩 (GPT-4o 사용)
llm = ChatOpenAI(model="gpt-4o", api_key=openai_key, temperature=0) 
llm_with_tools = llm.bind_tools(
    tools=[wiki_summary, db_search_cafe_func] # Tavily 제외, 2개 도구 바인딩
)

# 도구 동적 실행을 위한 매핑
available_tools: Dict[str, Any] = {
    "wiki_summary": wiki_summary,
    "db_search_cafe_func": db_search_cafe_func
}



In [12]:

# ----------------------------------------------------------------------
# 3. 간단한 도구 호출 체인 구현 (@chain 데코레이터 사용) - 오류 수정 버전
# ----------------------------------------------------------------------

@chain
def tool_calling_chain(question: str):
    """
    사용자 질문을 받아 LLM이 도구를 선택하고, 도구 호출 결과를 처리하여 최종 답변을 생성하는 체인.
    """
    # 1. LLM에게 질문 전달 및 도구 호출 결정 유도
    ai_message = llm_with_tools.invoke([HumanMessage(content=question)])

    # 2. 도구 호출이 없는 경우 (일반 응답)
    if not ai_message.tool_calls:
        return ai_message.content

    tool_messages = []
    
    print("\n--- LLM의 도구 호출 결정 ---")
    pprint(ai_message.tool_calls)
    
    # 3. 도구 호출 실행 (TypeError 방지 로직 적용)
    for tool_call in ai_message.tool_calls:
        tool_name = tool_call["name"]
        tool_call_id = tool_call["id"]
        
        # ⚠️ 핵심 수정: tool_call["args"]가 str인지 dict인지 확인하여 처리
        args_data = tool_call["args"]
        
        if isinstance(args_data, str):
            # 문자열인 경우에만 JSON 파싱 시도 (이전 버전/모델 호환)
            try:
                tool_args = json.loads(args_data)
            except json.JSONDecodeError:
                print(f"[경고] 도구 {tool_name}의 인자 파싱 오류: {args_data}")
                tool_args = {} 
        else:
            # dict인 경우 그대로 사용 (최신 LangChain/OpenAI 호환)
            tool_args = args_data 

        print(f"[Tool Execution] {tool_name} 호출, 인자: {tool_args}")
        
        # 도구 동적 실행
        if tool_name in available_tools:
            tool_func = available_tools[tool_name]
            # 도구의 invoke 메서드에 인자를 전달하여 실행
            result = tool_func.invoke(tool_args)
        else:
            result = f"알 수 없는 도구: {tool_name}"
            
        # 4. 도구 실행 결과를 ToolMessage로 래핑
        tool_messages.append(
            ToolMessage(content=str(result), tool_call_id=tool_call_id)
        )

    # 5. 도구 실행 결과와 LLM 응답을 다시 LLM에 전달하여 최종 답변 생성
    # (HumanMessage(content=question)은 중복되므로 제거하고 ai_message부터 시작)
    final_response = llm.invoke([
        ai_message,
        *tool_messages
    ])
    
    return final_response.content



In [13]:

# ----------------------------------------------------------------------
# 4. 테스트 질문 처리
# ----------------------------------------------------------------------
print("\n" + "=" * 80)
print(">>> 4단계: RAG 체인 테스트 시작")
print("=" * 80)

# 테스트 질문
test_question_1 = "아메리카노의 가격과 특징은 무엇인가요?" # -> db_search_cafe_func 사용 예상
test_question_2 = "커피의 기원에 대해 알려줄 수 있니?" # -> wiki_summary 사용 예상
test_question_3 = "바닐라 라떼와 초코 프라페의 재료와 가격을 비교해줘." # -> db_search_cafe_func 사용 예상

for i, question in enumerate([test_question_1, test_question_2, test_question_3]):
    print(f"\n[테스트 질문 {i+1}] {question}")
    
    final_answer = tool_calling_chain.invoke(question)
    
    print("\n--- AI 최종 답변 ---")
    print(final_answer)
    print("--------------------")

print("\n" + "=" * 80)
print(">>> 테스트 완료")


>>> 4단계: RAG 체인 테스트 시작

[테스트 질문 1] 아메리카노의 가격과 특징은 무엇인가요?

--- LLM의 도구 호출 결정 ---
[{'args': {'query': '아메리카노 가격'},
  'id': 'call_0QDEBOSmfmfviyGoZvOKmcQc',
  'name': 'db_search_cafe_func',
  'type': 'tool_call'},
 {'args': {'topic': '아메리카노 특징'},
  'id': 'call_LCLvobOZ8oTqThwov9Ph5TyZ',
  'name': 'wiki_summary',
  'type': 'tool_call'}]
[Tool Execution] db_search_cafe_func 호출, 인자: {'query': '아메리카노 가격'}

[Tool Call] 로컬 메뉴 DB 검색 실행: 아메리카노 가격


C:\Users\User\AppData\Local\Temp\ipykernel_5972\543531210.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = cafe_retriever.get_relevant_documents(query)


[Tool Execution] wiki_summary 호출, 인자: {'topic': '아메리카노 특징'}

[Tool Call] 위키피디아 검색 실행: 아메리카노 특징

--- AI 최종 답변 ---
아메리카노는 에스프레소를 뜨거운 물로 희석하여 마시는 커피 음료의 한 종류입니다. 일반적인 드립 커피와 비슷하지만, 풍미는 다릅니다. 아메리카노의 농도는 에스프레소의 '샷' 수와 더해지는 물의 양에 따라 달라집니다.

다음은 몇 가지 카페에서의 아메리카노 가격 정보입니다:

1. **카페 아메리카노**: 가격은 ₩4,500이며, 에스프레소와 뜨거운 물로 구성되어 있습니다. 원두 본연의 깔끔하고 시원한 맛이 특징입니다.

2. **카페 라떼**: 가격은 ₩5,000이며, 에스프레소와 우유로 구성되어 있습니다. 우유의 고소함과 에스프레소의 조화가 부드럽습니다.

3. **바닐라 라떼**: 가격은 ₩5,500이며, 에스프레소, 우유, 바닐라 시럽으로 구성되어 있습니다. 달콤한 바닐라 향이 추가된 부드러운 라떼입니다.

이 정보는 특정 카페의 메뉴를 기반으로 한 것이며, 실제 가격은 카페에 따라 다를 수 있습니다.
--------------------

[테스트 질문 2] 커피의 기원에 대해 알려줄 수 있니?

--- LLM의 도구 호출 결정 ---
[{'args': {'topic': '커피의 기원'},
  'id': 'call_QdPSzeZ3MESqXuEJRljaJJLG',
  'name': 'wiki_summary',
  'type': 'tool_call'}]
[Tool Execution] wiki_summary 호출, 인자: {'topic': '커피의 기원'}

[Tool Call] 위키피디아 검색 실행: 커피의 기원

--- AI 최종 답변 ---
커피의 기원은 동아프리카에서 시작되었으며, 이후 중동, 유럽, 인도 등으로 퍼지면서 오늘날 전 세계적으로 널리 소비되고 있습니다. "커피"라는 단어는 아랍어 "카흐와(قهوة)"에서 유래했으며, 이는 오스만 터키어 "